In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import keras

from keras.models import Sequential
from keras.models import load_model

from keras.layers import Flatten, Dense
from keras.layers import BatchNormalization
from keras.layers import Conv2D, Dropout

from keras.optimizers import SGD
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint
import tensorflow.compat.v1 as tf
from PIL import Image
from pandas import read_csv
import io
train_file = read_csv('train.csv')
train_file = train_file.iloc[:, 1:]
valid_file = read_csv('valid.csv')
valid_file=valid_file.iloc[:, 1:]

img = Image.open('mm/'+ str(39352) + '.png')
img =img.convert('RGB')
width, height = img.size
left = 0
top = height / 6
right = 320
bottom = height
img = img.crop((left, top, right, bottom))
img = img.resize((200,66))
arr = image.img_to_array(img)
input_shape=arr.shape
OUTPUT_NORMALIZATION = 66

In [6]:
train_file.shape

(25554, 2)

In [7]:
wheel_axis1 = valid_file['steer']    
img_list1 = valid_file['nombre']
X_valid=np.zeros((len(img_list1),) + arr.shape)
y_valid=np.zeros((len(img_list1),1))
for i in range(len(img_list1)):
            img_name = img_list1[i]
            img = Image.open('mm/'+ str(int(img_name)) + '.png')
            img =img.convert('RGB')
            left = 0
            top = height / 6
            right = 320
            bottom = height
            img = img.crop((left, top, right, bottom))
            img = img.resize((200,66))
            arr = image.img_to_array(img)
            y_valid[i] = wheel_axis1[i]/ OUTPUT_NORMALIZATION    
            X_valid[i] = arr/255


In [8]:
wheel_axis = train_file['steer']    
img_list = train_file['nombre']
X_train=np.zeros((len(img_list),) + arr.shape)
y_train=np.zeros((len(img_list),1))
for i in range(len(img_list)):
            img_name = img_list[i]
            img = Image.open('mm/'+ str(int(img_name)) + '.png')
            img =img.convert('RGB')
            left = 0
            top = height / 6
            right = 320
            bottom = height
            img = img.crop((left, top, right, bottom))
            img = img.resize((200,66))
            arr = image.img_to_array(img)
            y_train[i] = wheel_axis[i]/ OUTPUT_NORMALIZATION    
            X_train[i] = arr/255
            

batch_size = 80

In [9]:
def get_model(input_shape):
    model = Sequential([
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu', input_shape=input_shape),
        BatchNormalization(axis=1),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.2),
        BatchNormalization(),
        Dense(50, activation='relu'),
        Dropout(0.2),
        BatchNormalization(),
        Dense(10, activation='relu'),
        Dropout(0.2),
        BatchNormalization(),
        Dense(1)
    ])
    
    return model

model = get_model(input_shape)
sgd = SGD(lr=1e-3, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss="mse") 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
batch_normalization_1 (Batch (None, 31, 98, 24)        124       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 47, 36)        56        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
batch_normalization_3 (Batch (None, 5, 22, 48)         20        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 20, 64)        

In [ ]:
from keras.callbacks import LearningRateScheduler, EarlyStopping, TerminateOnNaN,History
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, label_ranking_average_precision_score, label_ranking_loss, coverage_error 


history = model.fit(X_train, y_train, 
                    epochs=20, 
                    batch_size=batch_size, 
                    verbose=2, 
                    validation_data=(X_valid, y_valid),
                    shuffle=True)    
model.save('model.h5') 
y_pred = model.predict(X_valid, batch_size=batch_size)

print(classification_report(y_valid.argmax(axis=1), y_pred.argmax(axis=1)))

Train on 25554 samples, validate on 6389 samples
Epoch 1/20


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib import image
from os import listdir
import math



loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(loss))
 
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()